In [1]:
import numpy as np
import pandas as pd

In [2]:
elsa = pd.read_csv(r'C:\Users\s1989190\Documents\GitHub\Structure-of-Frailty-ELSA\Analysis\Data\elsa_wave9.csv')
deficits = elsa.loc[:, 'mob-walk100m': 'mem_day']
deficits = deficits.dropna() # Remove rows with missing values

In [4]:
import numpy as np

def boolmf(X, r, options=None):
    m, n = X.shape
    if options is None:
        options = {}
    if 'P' not in options:
        options['P'] = np.ones((m, n))
    if 'init' not in options:
        options['init'] = 1
    if 'iterBoolMF' not in options:
        options['iterBoolMF'] = 100
    if 'timeBoolLS' not in options:
        options['timeBoolLS'] = 10
    
    if np.sum(np.logical_or(X == 0, X == 1)) < m * n:
        print('The input matrix should be binary although the code works for non-binary inputs.')

    if 'W' not in options:
        if options['init'] == 1:
            W = boolmf_init_wnmf(X, r, options['P'])
        elif options['init'] == 2:
            W = boolmf_init_rand_x(X, r, options)
    else:
        W = options['W']
        if W.shape != (m, r):
            raise ValueError('The input options.W does not have the right dimensions.')
        if np.sum(np.logical_or(W == 0, W == 1)) < m * r:
            print('The input options.W is not binary.')

    eprev = float('inf')
    enew = 0
    i = 1
    while eprev - enew > 0 and i <= options['iterBoolMF']:
        eprev = enew
        H = exact_boolean_matrix_ls(X, W, options['P'], options['timeBoolLS'])
        if i == 1:
            eprev = np.round(np.sum(options['P'] * ((X - np.minimum(1, W @ H)) ** 2)))
        
        ind0 = np.where(np.sum(H, axis=1) == 0)[0]
        if len(ind0) > 0:
            R = np.maximum(0, X - np.minimum(1, W @ H))
            sumR = np.sum(R, axis=1)
            b = np.argsort(-R, axis=1)
            H[ind0, :] = R[np.arange(len(ind0))[:, np.newaxis], b[ind0, :][:, :len(ind0)]]
        
        W = np.transpose(exact_boolean_matrix_ls(np.transpose(X), np.transpose(H), np.transpose(options['P']), options['timeBoolLS']))
        enew = np.round(np.sum(options['P'] * ((X - np.minimum(1, W @ H)) ** 2)))
        i += 1
    
    i -= 1
    return W, H, enew, i

# Define boolmf_init_wnmf, boolmf_init_rand_x, and exact_boolean_matrix_ls functions
# as per your specific implementation.
